## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns={'EIN', 'NAME'})

In [3]:
# Determine the number of unique values in each column.
print(application_df.nunique())

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
app_numbers = application_df['APPLICATION_TYPE'].value_counts()
app_numbers

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace= list(app_numbers[app_numbers < 500].index)

In [7]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
classification_numbers = application_df["CLASSIFICATION"].value_counts()
classification_numbers

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2600        1
C6100        1
C5200        1
C1245        1
C2500        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_numbers = classification_numbers[classification_numbers >= 1]
classification_numbers

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2600        1
C6100        1
C5200        1
C1245        1
C2500        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_numbers[classification_numbers < 1800].index)

In [11]:
# Replace in dataframe
for cls in classifications_to_replace:
    application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(cls, "Other")
    
# Check to make sure bucketing was successful
application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_df=pd.get_dummies(application_df)
dummy_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [13]:
# Split our preprocessed data into our features and target arrays
X=dummy_df.drop(columns={'IS_SUCCESSFUL'}, axis=1).values
y=dummy_df['IS_SUCCESSFUL'].values.reshape(-1, 1)
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 2
hidden_nodes_layer2 = 2

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))  # binary clasifier -> 1 output neuron/sigmoid activation

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2)                 88        
                                                                 
 dense_1 (Dense)             (None, 2)                 6         
                                                                 
 dense_2 (Dense)             (None, 1)                 3         
                                                                 
Total params: 97
Trainable params: 97
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
# Add model weight checkpoints during training
import os
from tensorflow.keras.callbacks import ModelCheckpoint
os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path="checkpoints/weights.{epoch}.hdf5"

# Create callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              verbose=1,
                              save_weights_only=True,
                              save_freq=5
                              )

In [18]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=50, callbacks=[cp_callback]) 

Epoch 1/50
  1/804 [..............................] - ETA: 5:31 - loss: 0.6931 - accuracy: 0.2812
Epoch 00001: saving model to checkpoints\weights.1.hdf5

Epoch 00001: saving model to checkpoints\weights.1.hdf5
 10/804 [..............................] - ETA: 5s - loss: 0.6929 - accuracy: 0.5406  
Epoch 00001: saving model to checkpoints\weights.1.hdf5

Epoch 00001: saving model to checkpoints\weights.1.hdf5
 22/804 [..............................] - ETA: 4s - loss: 0.6928 - accuracy: 0.5398
Epoch 00001: saving model to checkpoints\weights.1.hdf5

Epoch 00001: saving model to checkpoints\weights.1.hdf5
 33/804 [>.............................] - ETA: 3s - loss: 0.6928 - accuracy: 0.5350
Epoch 00001: saving model to checkpoints\weights.1.hdf5

Epoch 00001: saving model to checkpoints\weights.1.hdf5
 41/804 [>.............................] - ETA: 4s - loss: 0.6927 - accuracy: 0.5373
Epoch 00001: saving model to checkpoints\weights.1.hdf5

Epoch 00001: saving model to checkpoints\weights.1.

410/804 [==============>...............] - ETA: 2s - loss: 0.6916 - accuracy: 0.5350
Epoch 00001: saving model to checkpoints\weights.1.hdf5

Epoch 00001: saving model to checkpoints\weights.1.hdf5
420/804 [==============>...............] - ETA: 2s - loss: 0.6917 - accuracy: 0.5342
Epoch 00001: saving model to checkpoints\weights.1.hdf5

Epoch 00001: saving model to checkpoints\weights.1.hdf5
430/804 [===============>..............] - ETA: 2s - loss: 0.6918 - accuracy: 0.5328
Epoch 00001: saving model to checkpoints\weights.1.hdf5

Epoch 00001: saving model to checkpoints\weights.1.hdf5
440/804 [===============>..............] - ETA: 2s - loss: 0.6918 - accuracy: 0.5330
Epoch 00001: saving model to checkpoints\weights.1.hdf5

Epoch 00001: saving model to checkpoints\weights.1.hdf5
450/804 [===============>..............] - ETA: 2s - loss: 0.6918 - accuracy: 0.5328
Epoch 00001: saving model to checkpoints\weights.1.hdf5

Epoch 00001: saving model to checkpoints\weights.1.hdf5
460/804 [=


Epoch 00002: saving model to checkpoints\weights.2.hdf5
 21/804 [..............................] - ETA: 4s - loss: 0.6937 - accuracy: 0.5104
Epoch 00002: saving model to checkpoints\weights.2.hdf5

Epoch 00002: saving model to checkpoints\weights.2.hdf5
 34/804 [>.............................] - ETA: 3s - loss: 0.6913 - accuracy: 0.5303
Epoch 00002: saving model to checkpoints\weights.2.hdf5

Epoch 00002: saving model to checkpoints\weights.2.hdf5
 45/804 [>.............................] - ETA: 3s - loss: 0.6924 - accuracy: 0.5215
Epoch 00002: saving model to checkpoints\weights.2.hdf5

Epoch 00002: saving model to checkpoints\weights.2.hdf5
 54/804 [=>............................] - ETA: 3s - loss: 0.6922 - accuracy: 0.5226
Epoch 00002: saving model to checkpoints\weights.2.hdf5

Epoch 00002: saving model to checkpoints\weights.2.hdf5

Epoch 00002: saving model to checkpoints\weights.2.hdf5
 66/804 [=>............................] - ETA: 3s - loss: 0.6919 - accuracy: 0.5251
Epoch 000

436/804 [===============>..............] - ETA: 1s - loss: 0.6915 - accuracy: 0.5287
Epoch 00002: saving model to checkpoints\weights.2.hdf5

Epoch 00002: saving model to checkpoints\weights.2.hdf5
446/804 [===============>..............] - ETA: 1s - loss: 0.6914 - accuracy: 0.5295
Epoch 00002: saving model to checkpoints\weights.2.hdf5

Epoch 00002: saving model to checkpoints\weights.2.hdf5
456/804 [================>.............] - ETA: 1s - loss: 0.6914 - accuracy: 0.5300
Epoch 00002: saving model to checkpoints\weights.2.hdf5

Epoch 00002: saving model to checkpoints\weights.2.hdf5
466/804 [================>.............] - ETA: 1s - loss: 0.6913 - accuracy: 0.5305
Epoch 00002: saving model to checkpoints\weights.2.hdf5

Epoch 00002: saving model to checkpoints\weights.2.hdf5
476/804 [================>.............] - ETA: 1s - loss: 0.6913 - accuracy: 0.5305
Epoch 00002: saving model to checkpoints\weights.2.hdf5

Epoch 00002: saving model to checkpoints\weights.2.hdf5
486/804 [=

 37/804 [>.............................] - ETA: 4s - loss: 0.6904 - accuracy: 0.5372
Epoch 00003: saving model to checkpoints\weights.3.hdf5

Epoch 00003: saving model to checkpoints\weights.3.hdf5
 47/804 [>.............................] - ETA: 4s - loss: 0.6916 - accuracy: 0.5293
Epoch 00003: saving model to checkpoints\weights.3.hdf5

Epoch 00003: saving model to checkpoints\weights.3.hdf5
 57/804 [=>............................] - ETA: 4s - loss: 0.6906 - accuracy: 0.5362
Epoch 00003: saving model to checkpoints\weights.3.hdf5

Epoch 00003: saving model to checkpoints\weights.3.hdf5
 67/804 [=>............................] - ETA: 4s - loss: 0.6909 - accuracy: 0.5340
Epoch 00003: saving model to checkpoints\weights.3.hdf5

Epoch 00003: saving model to checkpoints\weights.3.hdf5
 77/804 [=>............................] - ETA: 4s - loss: 0.6895 - accuracy: 0.5434
Epoch 00003: saving model to checkpoints\weights.3.hdf5

Epoch 00003: saving model to checkpoints\weights.3.hdf5
 87/804 [=

...

547/804 [===================>..........] - ETA: 1s - loss: 0.6909 - accuracy: 0.5337
Epoch 00048: saving model to checkpoints\weights.48.hdf5

Epoch 00048: saving model to checkpoints\weights.48.hdf5
557/804 [===================>..........] - ETA: 1s - loss: 0.6909 - accuracy: 0.5339
Epoch 00048: saving model to checkpoints\weights.48.hdf5

Epoch 00048: saving model to checkpoints\weights.48.hdf5
567/804 [====================>.........] - ETA: 1s - loss: 0.6910 - accuracy: 0.5333
Epoch 00048: saving model to checkpoints\weights.48.hdf5

Epoch 00048: saving model to checkpoints\weights.48.hdf5
577/804 [====================>.........] - ETA: 1s - loss: 0.6909 - accuracy: 0.5341
Epoch 00048: saving model to checkpoints\weights.48.hdf5

Epoch 00048: saving model to checkpoints\weights.48.hdf5
588/804 [====================>.........] - ETA: 1s - loss: 0.6910 - accuracy: 0.5330
Epoch 00048: saving model to checkpoints\weights.48.hdf5

Epoch 00048: saving model to checkpoints\weights.48.hdf5


148/804 [====>.........................] - ETA: 3s - loss: 0.6912 - accuracy: 0.5315
Epoch 00049: saving model to checkpoints\weights.49.hdf5

Epoch 00049: saving model to checkpoints\weights.49.hdf5
158/804 [====>.........................] - ETA: 3s - loss: 0.6913 - accuracy: 0.5307
Epoch 00049: saving model to checkpoints\weights.49.hdf5

Epoch 00049: saving model to checkpoints\weights.49.hdf5
168/804 [=====>........................] - ETA: 3s - loss: 0.6915 - accuracy: 0.5292
Epoch 00049: saving model to checkpoints\weights.49.hdf5

Epoch 00049: saving model to checkpoints\weights.49.hdf5
178/804 [=====>........................] - ETA: 3s - loss: 0.6912 - accuracy: 0.5320
Epoch 00049: saving model to checkpoints\weights.49.hdf5

Epoch 00049: saving model to checkpoints\weights.49.hdf5
188/804 [======>.......................] - ETA: 3s - loss: 0.6911 - accuracy: 0.5321
Epoch 00049: saving model to checkpoints\weights.49.hdf5

Epoch 00049: saving model to checkpoints\weights.49.hdf5


553/804 [===================>..........] - ETA: 1s - loss: 0.6912 - accuracy: 0.5314
Epoch 00049: saving model to checkpoints\weights.49.hdf5

Epoch 00049: saving model to checkpoints\weights.49.hdf5
563/804 [====================>.........] - ETA: 1s - loss: 0.6912 - accuracy: 0.5317
Epoch 00049: saving model to checkpoints\weights.49.hdf5

Epoch 00049: saving model to checkpoints\weights.49.hdf5
573/804 [====================>.........] - ETA: 1s - loss: 0.6912 - accuracy: 0.5317
Epoch 00049: saving model to checkpoints\weights.49.hdf5

Epoch 00049: saving model to checkpoints\weights.49.hdf5
583/804 [====================>.........] - ETA: 1s - loss: 0.6911 - accuracy: 0.5321
Epoch 00049: saving model to checkpoints\weights.49.hdf5

Epoch 00049: saving model to checkpoints\weights.49.hdf5
593/804 [=====================>........] - ETA: 1s - loss: 0.6911 - accuracy: 0.5324
Epoch 00049: saving model to checkpoints\weights.49.hdf5

Epoch 00049: saving model to checkpoints\weights.49.hdf5



Epoch 00050: saving model to checkpoints\weights.50.hdf5
159/804 [====>.........................] - ETA: 3s - loss: 0.6896 - accuracy: 0.5423
Epoch 00050: saving model to checkpoints\weights.50.hdf5

Epoch 00050: saving model to checkpoints\weights.50.hdf5
169/804 [=====>........................] - ETA: 3s - loss: 0.6893 - accuracy: 0.5444
Epoch 00050: saving model to checkpoints\weights.50.hdf5

Epoch 00050: saving model to checkpoints\weights.50.hdf5
179/804 [=====>........................] - ETA: 3s - loss: 0.6895 - accuracy: 0.5429
Epoch 00050: saving model to checkpoints\weights.50.hdf5

Epoch 00050: saving model to checkpoints\weights.50.hdf5
189/804 [======>.......................] - ETA: 3s - loss: 0.6897 - accuracy: 0.5422
Epoch 00050: saving model to checkpoints\weights.50.hdf5

Epoch 00050: saving model to checkpoints\weights.50.hdf5
199/804 [======>.......................] - ETA: 3s - loss: 0.6897 - accuracy: 0.5421
Epoch 00050: saving model to checkpoints\weights.50.hdf5


564/804 [====================>.........] - ETA: 1s - loss: 0.6906 - accuracy: 0.5357
Epoch 00050: saving model to checkpoints\weights.50.hdf5

Epoch 00050: saving model to checkpoints\weights.50.hdf5
574/804 [====================>.........] - ETA: 1s - loss: 0.6907 - accuracy: 0.5351
Epoch 00050: saving model to checkpoints\weights.50.hdf5

Epoch 00050: saving model to checkpoints\weights.50.hdf5
584/804 [====================>.........] - ETA: 1s - loss: 0.6907 - accuracy: 0.5351
Epoch 00050: saving model to checkpoints\weights.50.hdf5

Epoch 00050: saving model to checkpoints\weights.50.hdf5
594/804 [=====================>........] - ETA: 1s - loss: 0.6907 - accuracy: 0.5350
Epoch 00050: saving model to checkpoints\weights.50.hdf5

Epoch 00050: saving model to checkpoints\weights.50.hdf5
604/804 [=====================>........] - ETA: 1s - loss: 0.6906 - accuracy: 0.5360
Epoch 00050: saving model to checkpoints\weights.50.hdf5

Epoch 00050: saving model to checkpoints\weights.50.hdf5


In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6918 - accuracy: 0.5266 - 346ms/epoch - 1ms/step
Loss: 0.6918013095855713, Accuracy: 0.5266472101211548


In [ ]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE